In [1]:
import util
from test_framework.address import program_to_witness
from test_framework.key import generate_key_pair
from test_framework.messages import COutPoint, CTransaction, CTxIn, CTxOut, CTxInWitness
from test_framework.script import SegwitV0SignatureHash, SIGHASH_ALL, hash160, get_p2pkh_script

Source directory configured as /workspaces/taproot-workshop/bitcoin


# 2.0 Taproot Introduction

Over the following chapters, we introduce various aspects of the schnorr/taproot soft fork proposal. Each chapter is built around a case study, demonstrating how the technology can be used in world applications.

In each chapter, we first introduce the case study scenario and give an overview of the technology. We then demonstrate its use as follows:

1. We first construct a segwit v1 witness program that implements the desired spending policy and derive the bech32m address for that witness program;
2. Then we start a Bitcoin Core full node, generate 101 blocks on the node (so that the node has a mature balance to spend), and spend coins from the Bitcoin Core wallet to the bech32m address from step (1);
3. Finally, we construct a transaction spending from the output created in step (2) back to the Bitcoin Core wallet. We sign the transaction and verify that the transaction is valid using the full node's `testmempoolaccept` RPC method.

This sequence of steps is illustrated below:

![test](images/segwit_version1_1.jpg)

In each chapter, we'll implement the spending policy using both v0 (pre-taproot) segwit and v1 (taproot) segwit outputs, and highlight the differences in transaction weight and privacy.

## Transaction sequence

This chapter demonstrates the transaction sequence in full detail. Future chapters follow the same steps, but use convenience functions to abstract away the low-level details.

#### Example 2.0.1 Generate a segwit v0 bech32 address

We generate an address *outside* the Bitcoin Core wallet, which we'll send funds to from Bitcoin Core.

In this example, we'll use a P2WPKH segwit output (not a taproot output).

In [2]:
# Generate a new key pair
privkey, pubkey = generate_key_pair()
print("Pubkey: {}\n".format(pubkey.get_bytes(bip340=False).hex()))

# Get the hash160 of the public key for the witness program
# Note that the function 'get_bytes(bip340=False)' is used to get the compressed DER encoding of the public key needed for
# segwit v0.
program = hash160(pubkey.get_bytes(bip340=False))
print("Witness program: {}\n".format(program.hex()))

# Create (regtest) bech32 address
version = 0x00
address = program_to_witness(version, program)
print("bech32 address: {}".format(address))

Pubkey: 03fdbc816dc3770c950e4ec3275242ef52212238bb4ae663edcbc63e424ad97bb7

Witness program: bb0d8d966e052c82d07d2290583a05b3c33543ce

bech32 address: bcrt1qhvxcm9nwq5kg95ray2g9sws9k0pn2s7w6kph8m


#### Example 2.0.2 Start a Bitcoin Core node, then generate blocks and send output to the bech32 address generated above

This functionality will be encapsulated in the `node.generate_and_send_coins(address)` method later.

In [3]:
# Start node
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

version = node.getnetworkinfo()['subversion']
print("\nClient version is {}\n".format(version))

# Generate 101 blocks
node.generate(101)
balance = node.getbalance()
print("Balance: {}\n".format(balance))

assert balance > 1

unspent_txid = node.listunspent(1)[-1]["txid"]
inputs = [{"txid": unspent_txid, "vout": 0}]

# Create a raw transaction sending 1 BTC to the address and then sign it.
tx_hex = node.createrawtransaction(inputs=inputs, outputs=[{address: 1}])
res = node.signrawtransactionwithwallet(hexstring=tx_hex)

tx_hex = res["hex"]
assert res["complete"]
assert 'errors' not in res

# Send the raw transaction. We haven't created a change output,
# so maxfeerate must be set to 0 to allow any fee rate.
txid = node.sendrawtransaction(hexstring=tx_hex, maxfeerate=0)

print("Transaction {}, output 0\nsent to {}".format(txid, address))

2024-05-23T14:49:34.797000Z TestFramework./tmp/bitcoin_func_test_ncp1pwv0 (INFO): Initializing test directory /tmp/bitcoin_func_test_ncp1pwv0

Client version is /Satoshi:22.1.0(testnode0)/

Balance: 50.00000000

Transaction 70be71ec36d8d0f698cd6a1e3df95c80746929c777fb2406d95c67f437a2ab45, output 0
sent to bcrt1qhvxcm9nwq5kg95ray2g9sws9k0pn2s7w6kph8m


#### Example 2.0.3 Construct a transaction to spend the coins back to the Bitcoin Core wallet

In this example, we'll manually construct a transaction which spends the output back to the Bitcoin Core wallet.

To do that we create a `CTransaction` object and populate the data members:

 * `nVersion`
 * `nLocktime`  
 * `tx_vin` (list of `CTxIn` objects)
 * `tx_vout` (list of `CTxOut` objects)

This functionality will be encapsulated in the `test.create_spending_transaction(coin_txid, version)` method later.

The only item that we don't populate is the witness:
 
 * `tx.wit.vtxinwit` (list of `CTxInWitness` objects)

which we'll do later.

In [4]:
# Construct transaction
spending_tx = CTransaction()

# Populate the transaction version
spending_tx.nVersion = 1

# Populate the locktime
spending_tx.nLockTime = 0

# Populate the transaction inputs
outpoint = COutPoint(int(txid, 16), 0)
spending_tx_in = CTxIn(outpoint)
spending_tx.vin = [spending_tx_in]

# Generate new Bitcoin Core wallet address
dest_addr = node.getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(node.getaddressinfo(dest_addr)['scriptPubKey'])

# Complete output which returns 0.5 BTC to Bitcoin Core wallet
amount_sat = int(0.5 * 100_000_000)
dest_output = CTxOut(nValue=amount_sat, scriptPubKey=scriptpubkey)
spending_tx.vout = [dest_output]

print("Spending transaction:\n{}".format(spending_tx))

Spending transaction:
CTransaction(nVersion=1 vin=[CTxIn(prevout=COutPoint(hash=70be71ec36d8d0f698cd6a1e3df95c80746929c777fb2406d95c67f437a2ab45 n=0) scriptSig= nSequence=0)] vout=[CTxOut(nValue=0.50000000 scriptPubKey=0014479817d95b6afe5be52996ad3eb8cf4dc68ce8d1)] wit=CTxWitness() nLockTime=0)


#### Example 2.0.4 Generate signature for transaction, add it to the witness, and test mempool acceptance.

In this example, we sign the transaction, add the signature to the transaction's witness, and then use `testmempoolaccept` to verify that the transaction is valid. Later on, we'll use the `node.test_transaction()` convenience method to test mempool acceptance.

In [5]:
# Generate the segwit v0 signature hash for signing
sighash = SegwitV0SignatureHash(script=get_p2pkh_script(program),
                                txTo=spending_tx,
                                inIdx=0,
                                hashtype=SIGHASH_ALL,
                                amount=100_000_000)

# Sign using ECDSA and append the SIGHASH byte
sig = privkey.sign_ecdsa(sighash) + chr(SIGHASH_ALL).encode('latin-1')

print("Signature: {}\n".format(sig.hex()))

# Add a witness to the transaction. For a P2WPKH, the witness field is the signature and pubkey
spending_tx.wit.vtxinwit.append(CTxInWitness([sig, pubkey.get_bytes(bip340=False)]))

print("Spending transaction:\n{}\n".format(spending_tx))

# Serialize signed transaction for broadcast
spending_tx_str = spending_tx.serialize().hex()

# Test mempool acceptance
assert node.testmempoolaccept(rawtxs=[spending_tx_str], maxfeerate=0)[0]['allowed']
print("Success!")

Signature: 304402202ed4d3c687cc581d0340ff11cded2ffb2b723d22db974aa3fb24a84878b981e70220246ec816901efd23d562b779374adc2ecc143c56c7307db52d2ad030624af66301

Spending transaction:
CTransaction(nVersion=1 vin=[CTxIn(prevout=COutPoint(hash=70be71ec36d8d0f698cd6a1e3df95c80746929c777fb2406d95c67f437a2ab45 n=0) scriptSig= nSequence=0)] vout=[CTxOut(nValue=0.50000000 scriptPubKey=0014479817d95b6afe5be52996ad3eb8cf4dc68ce8d1)] wit=CTxWitness(CScriptWitness(304402202ed4d3c687cc581d0340ff11cded2ffb2b723d22db974aa3fb24a84878b981e70220246ec816901efd23d562b779374adc2ecc143c56c7307db52d2ad030624af66301,03fdbc816dc3770c950e4ec3275242ef52212238bb4ae663edcbc63e424ad97bb7)) nLockTime=0)

Success!


#### Example 2.0.5 Shutdown the node

Finally we run `test.shutdown()` to end the test and shutdown the node.

In [6]:
test.shutdown()

2024-05-23T14:49:59.360000Z TestFramework./tmp/bitcoin_func_test_ncp1pwv0 (INFO): Stopping nodes
2024-05-23T14:49:59.412000Z TestFramework./tmp/bitcoin_func_test_ncp1pwv0 (INFO): Cleaning up /tmp/bitcoin_func_test_ncp1pwv0 on exit
2024-05-23T14:49:59.412000Z TestFramework./tmp/bitcoin_func_test_ncp1pwv0 (INFO): Tests successful


#### Example 2.0.6 Repeat the steps using convenience functions

We'll repeat the set of steps using the convenience functions:

1. Start the node and create an output that we can spend using `node.generate_and_send_coins(address)`
2. Create a `CTransaction` object that spends the output back to the Bitcoin Core wallet using `test.create_spending_transaction(coin_txid, version)`
3. Manually sign the transaction and add the witness
4. Test mempool acceptance using `node.test_transaction()`

In [7]:
# Start node
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

# Generate coins and create an output
tx = node.generate_and_send_coins(address)
print("Transaction {}, output 0\nsent to {}\n".format(tx.hash, address))

# Create a spending transaction
spending_tx = test.create_spending_transaction(tx.hash)
print("Spending transaction:\n{}\n".format(spending_tx))

# Sign the spending transaction and append the witness
sighash = SegwitV0SignatureHash(script=get_p2pkh_script(program),
                                txTo=spending_tx,
                                inIdx=0,
                                hashtype=SIGHASH_ALL,
                                amount=100_000_000)
sig = privkey.sign_ecdsa(sighash) + chr(SIGHASH_ALL).encode('latin-1')
spending_tx.wit.vtxinwit.append(CTxInWitness([sig, pubkey.get_bytes(bip340=False)]))

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")

2024-05-23T14:50:06.263000Z TestFramework./tmp/bitcoin_func_test_idkuuxv6 (INFO): Initializing test directory /tmp/bitcoin_func_test_idkuuxv6

Client version is /Satoshi:22.1.0(testnode0)/

Balance: 50.00000000

Transaction 38d0a737bb5f452e482b95334c63b6a2e0e29a8ceae5e51f1078580c02311086, output 0
sent to bcrt1qhvxcm9nwq5kg95ray2g9sws9k0pn2s7w6kph8m

Spending transaction:
CTransaction(nVersion=1 vin=[CTxIn(prevout=COutPoint(hash=38d0a737bb5f452e482b95334c63b6a2e0e29a8ceae5e51f1078580c02311086 n=0) scriptSig= nSequence=0)] vout=[CTxOut(nValue=0.50000000 scriptPubKey=00143ca78782cc631ac869713faa3871d85e05be8a40)] wit=CTxWitness() nLockTime=0)

{'txid': '9499e1a75ca047a37a6b11808dbc4f88e6467c86ef6faaa580d375991f046084', 'wtxid': '4d88d6ffa3e987895c3488df63a8c5b34f8cf3eddef2960d312efeef74961a9f', 'allowed': True, 'vsize': 110, 'fees': {'base': Decimal('0.50000000')}}
Success!


#### Example 2.0.7 Shutdown the node

As always, we finish by shutting down the node.

In [8]:
test.shutdown()

2024-05-23T14:50:11.580000Z TestFramework./tmp/bitcoin_func_test_idkuuxv6 (INFO): Stopping nodes
2024-05-23T14:50:11.633000Z TestFramework./tmp/bitcoin_func_test_idkuuxv6 (INFO): Cleaning up /tmp/bitcoin_func_test_idkuuxv6 on exit
2024-05-23T14:50:11.633000Z TestFramework./tmp/bitcoin_func_test_idkuuxv6 (INFO): Tests successful


**Congratulations!** In this chapter, you have:

- Started a Bitcoin Core full node (in regtest mode), generated 101 blocks and sent a transaction output to a segwit address
- Constructed a transaction that spends the segwit output back to the wallet, and tested that it is accepted by the mempool
- Repeated the same steps using the `generate_and_send_coins()` and `create_spending_transaction()` convenience functions

We'll use exactly the same sequence of steps in future chapters to spend to and from segwit v1 addresses.